# Quasi static forces from many model tests

# Purpose
Calculating the quasi static forces from a model test by assuming the added masses. This can then be compared to the static VCT calculations.

# Setup

In [ ]:
# %load imports.py
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl
#import vessel_manoeuvring_models.models.nonlinear_martin_vmm as vmm
#import vessel_manoeuvring_models.nonlinear_martin_vmm_equations as eq

#import vessel_manoeuvring_models.models.linear_vmm as vmm
#import vessel_manoeuvring_models.nonlinear_vmm_equations as eq
#import vessel_manoeuvring_models.nonlinear_martin_vmm_equations as eq
import vessel_manoeuvring_models.nonlinear_abkowitz_vmm_equations as eq
from vessel_manoeuvring_models.models.vmm import Simulation

#import vessel_manoeuvring_models.models.linear_vmm as model
from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.plot import track_plot
from vessel_manoeuvring_models.equation import Equation

# Methodology
Generally:

$ F = m \cdot a $

if $m$ (including added mass) can is known $F$ should be possible to calculate from a dynamic model test.

For the ship in the Y-direction this can be written as:

## Y:

In [ ]:
Math(vlatex(eq.Y_eom))

In [ ]:
Math(vlatex(eq.fy_eq))

In [ ]:
Y_eq = eq.Y_eom.subs(eq.Y_force,sp.solve(eq.fy_eq,Y_force)[0])

#Y_eq = Y_eq.subs([
#    (eq.p.Yudot,0),
#    (eq.p.Yrdot,0),
#])

Math(vlatex(Y_eq))

In [ ]:
eq_Y_qs = sp.Eq(Y_qs,
      sp.solve(Y_eq, Y_qs)[0])
Math(vlatex(eq_Y_qs))

In [ ]:
subs = {value:key for key,value in eq.p.items()}
Y_qs_lambda = lambdify(eq_Y_qs.subs(subs).rhs)

## X:

In [ ]:
Math(vlatex(eq.X_eom))

In [ ]:
Math(vlatex(eq.fx_eq))

In [ ]:
X_eq = eq.X_eom.subs(eq.X_force,sp.solve(eq.fx_eq,X_force)[0])

#X_eq = X_eq.subs([
#    (eq.p.Xvdot,0),
#    (eq.p.Xrdot,0),
#])


Math(vlatex(X_eq))

In [ ]:
eq_X_qs = sp.Eq(X_qs,
      sp.solve(X_eq, X_qs)[0])
Math(vlatex(eq_X_qs))

In [ ]:
subs = {value:key for key,value in eq.p.items()}
X_qs_lambda = lambdify(eq_X_qs.subs(subs).rhs)

## N:

In [ ]:
Math(vlatex(eq.N_eom))

In [ ]:
Math(vlatex(eq.mz_eq))

In [ ]:
N_eq = eq.N_eom.subs(eq.N_force,sp.solve(eq.mz_eq,N_force)[0])

#N_eq = N_eq.subs([
#    (eq.p.Nudot,0),
#    (eq.p.Nvdot,0),
#])

Math(vlatex(N_eq))

In [ ]:
eq_N_qs = sp.Eq(N_qs,
      sp.solve(N_eq, N_qs)[0])
Math(vlatex(eq_N_qs))

In [ ]:
N_qs_lambda = lambdify(eq_N_qs.subs(subs).rhs)

In [ ]:
simulation = Simulation(X_eq=X_eq, Y_eq=Y_eq, N_eq=N_eq)

In [ ]:
simulation.A

In [ ]:
simulation.b

# Quasi static forces from model tests

## Load tests

In [ ]:
df_runs = df_runs = pd.read_csv('../data/processed/kalman_cut/runs.csv', index_col=0)
df_runs_selected = df_runs

In [ ]:
df_runs['test_type'].unique()

In [ ]:
#id=22773
#id=22616
#id=22774
##id=22770

def load_run(id):
    
    df, units, meta_data = mdl.load(id=id, dir_path='../data/processed/kalman_cut')
    df.index = df.index.total_seconds()
    df.index-=df.index[0]
    df.sort_index(inplace=True)
    df['-delta'] = -df['delta']
    df['V'] = np.sqrt(df['u']**2 + df['v']**2)
    
    return df, units, meta_data

df_all = pd.DataFrame()
for id,row in df_runs_selected.iterrows():
    
    df_, units, meta_data = load_run(id)
    df_['id'] = id
    df_['t'] = df_.index
    df_all = df_all.append(df_, ignore_index=True)

    
df_all['thrust'] = df_all['Prop/PS/Thrust'] + df_all['Prop/SB/Thrust']
df_all['U'] = df_all['V']

#Switch to infiltered:
#gradients = ['u','v','r','u1d','v1d','r1d']
#for gradient in gradients:
#    df_all[gradient] = df_all[f'{gradient}_gradient']
       

In [ ]:
fig,ax=plt.subplots()
df_all.plot(y='v_gradient',ax=ax)
df_all.plot(y='v', ax=ax)

fig,ax=plt.subplots()
df_all.plot(y='v1d_gradient',ax=ax)
df_all.plot(y='v1d', ax=ax)

In [ ]:
fig,ax=plt.subplots()
df_all.plot(y='v1d', ax=ax)

In [ ]:
runs = df_all.groupby(by='id')
for (test_type,ship_speed), group in df_runs_selected.groupby(by=['test_type','ship_speed']):
    
    fig,ax=plt.subplots()
    for id,_ in group.iterrows(): 
        df = runs.get_group(id)
        track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);
        
    ax.set_title(f'{test_type}: {ship_speed} kts')

In [ ]:
df_runs_ref_speed = df_runs_selected.groupby(by='test_type').get_group('reference speed')
mask = df_all['id'].isin(df_runs_ref_speed.index)
df_all_ref_speed = df_all.loc[mask]

In [ ]:
np.rad2deg(df_all_ref_speed['delta'].mean())

In [ ]:
meta_data['rho']=1000
meta_data['mass'] = meta_data['Volume']*meta_data['rho']
meta_data.dropna()

In [ ]:
df_all.head()

# Ship parameters

In [ ]:
T_ = (meta_data.TA + meta_data.TF)/2
L_ = meta_data.lpp
m_ = meta_data.mass
rho_ = meta_data.rho
B_ = meta_data.beam
CB_ = m_/(T_*B_*L_*rho_)
I_z_ = m_*meta_data.KZZ**2
#I_z_ = 900


ship_parameters = {
        'T' : T_,
        'L' : L_,
        'CB' :CB_,
        'B' : B_,
        'rho' : rho_,
        'x_G' : 0,  # motions are expressed at CG
        'm' : m_,
        'I_z': I_z_, 
        'volume':meta_data.Volume,
    }

ps = prime_system.PrimeSystem(**ship_parameters)  # model

scale_factor = meta_data.scale_factor
ps_ship = prime_system.PrimeSystem(L=ship_parameters['L']*scale_factor, rho=meta_data['rho'])  # ship

In [ ]:
ship_parameters

# Brix parameters

In [ ]:
def calculate_prime(row, ship_parameters):
    return run(function=row['brix_lambda'], inputs=ship_parameters)


mask = df_parameters['brix_lambda'].notnull()
df_parameters.loc[mask,'brix_prime'] = df_parameters.loc[mask].apply(calculate_prime, ship_parameters=ship_parameters, axis=1)

df_parameters['brix_prime'].fillna(0, inplace=True)
#df_parameters['brix_SI'].fillna(0, inplace=True)



In [ ]:
df_parameters.groupby(by='state').get_group('dot')

In [ ]:
df_parameters['prime'] = df_parameters['brix_prime']

def to_SI(row):
    
    prime = row['prime']
    denominator = row['denominator']
    lamda = lambdify(denominator)
    try:
        denominator_value = run(lamda, inputs=ship_parameters)
        SI = prime*denominator_value
    except:
        return None
    else:
        return SI
    
df_parameters['brix_SI'] = df_parameters.apply(to_SI, axis=1)
    

In [ ]:
df_all['Y_qs'] = run(Y_qs_lambda, inputs=ship_parameters, **df_all, **df_parameters['brix_SI'])
df_all['X_qs'] = run(X_qs_lambda, inputs=ship_parameters, **df_all, **df_parameters['brix_SI'])
df_all['N_qs'] = run(N_qs_lambda, inputs=ship_parameters, **df_all, **df_parameters['brix_SI'])

In [ ]:
Y_qs_lambda

In [ ]:
keys = ['X_qs','Y_qs','N_qs','delta']

for key in keys:
    title=key
    display(px.line(df_all, x='t', y=key, width=1000, height=400, line_group='id', color='id', title=title, hover_data=['id']))

In [ ]:
sns.pairplot(df_all, x_vars=['u','v','r','delta'], y_vars=['X_qs','Y_qs','N_qs'])

# Regression

## N

In [ ]:
Math(vlatex(eq.N_qs_eq))

In [ ]:
simulation.define_quasi_static_forces(X_qs_eq=eq.X_qs_eq, Y_qs_eq=eq.Y_qs_eq, N_qs_eq=eq.N_qs_eq)

In [ ]:
N_ = sp.symbols('N_')
diff_eq_N = regression.DiffEqToMatrix(ode=simulation.N_qs_eq.subs(N_qs,N_), 
                                      label=N_, base_features=[delta,u,v,r])

In [ ]:
diff_eq_N.acceleration_equation_x.rhs

In [ ]:
Math(vlatex(diff_eq_N.eq_X))

In [ ]:
diff_eq_N.eq_y

In [ ]:
diff_eq_N.y_lambda

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
def show_pred(X,y,results, label):
    
    display(results.summary())
    
    X_ = X
    y_ = y
    y_pred = results.predict(X_)
    
    prstd, iv_l, iv_u = wls_prediction_std(results, exog=X_, alpha=0.05)
    #iv_l*=-1 
    #iv_u*=-1
    
    fig,ax=plt.subplots()
    ax.plot(X_.index,y_, label='Numerical gradient from model test')
    ax.plot(X_.index,y_pred, '--', label='OLS')
    ax.set_ylabel(label)
    
    ax.fill_between(X_.index, y1=iv_l, y2=iv_u, zorder=-10, color='grey', alpha=0.5, label=r'5\% confidence')
    ax.legend();

In [ ]:
df_runs['test_type'].unique()

In [ ]:
mask = df_runs['test_type']!='reference speed'
df_runs_man = df_runs.loc[mask].copy()
mask = df_all['id'].isin(df_runs_man.index)
df_all_man = df_all.loc[mask]

In [ ]:
for id,group in df_all_man.groupby(by='id'):
    fig,ax=plt.subplots()
    track_plot(df=group, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);
    ax.set_title(id)

In [ ]:
X = diff_eq_N.calculate_features(data=df_all_man)
#X = sm.add_constant(X)
y = diff_eq_N.calculate_label(y=df_all_man['N_qs'])

model_N = sm.OLS(y,X)
results_N = model_N.fit()

show_pred(X=X,y=y,results=results_N, label=r'$N_{qs}$')

## Y

In [ ]:
Y_ = sp.symbols('Y_')
diff_eq_Y = regression.DiffEqToMatrix(ode=simulation.Y_qs_eq.subs(Y_qs,Y_), 
                                      label=Y_, base_features=[delta,u,v,r])

In [ ]:
X = diff_eq_Y.calculate_features(data=df_all_man)
#X = sm.add_constant(X)
y = diff_eq_Y.calculate_label(y=df_all_man['Y_qs'])

model_Y = sm.OLS(y,X)
results_Y = model_Y.fit()

show_pred(X=X,y=y,results=results_Y, label=r'$Y_{qs}$')

## X

In [ ]:
X_ = sp.symbols('X_')
diff_eq_X = regression.DiffEqToMatrix(ode=simulation.X_qs_eq.subs(X_qs,X_), 
                                      label=X_, base_features=[delta,u,v,r,thrust])

In [ ]:
X = diff_eq_X.calculate_features(data=df_all)
#X = sm.add_constant(X)
y = diff_eq_X.calculate_label(y=df_all['X_qs'])

model_X = sm.OLS(y,X)
results_X = model_X.fit()

show_pred(X=X,y=y,results=results_X, label=r'$X_{qs}$')

In [ ]:
results_summary_X = regression.results_summary_to_dataframe(results_X)
results_summary_Y = regression.results_summary_to_dataframe(results_Y)
results_summary_N = regression.results_summary_to_dataframe(results_N)

## Add the regressed parameters
Hydrodynamic derivatives that depend on acceleration cannot be obtained from the VCT regression. They are however essential if a time simulation should be conducted. These values have then been taken from Brix semi empirical formulas for the simulations below.

In [ ]:
df_parameters_all = df_parameters.copy()
for other in [results_summary_X, results_summary_Y, results_summary_N]:
    df_parameters_all = df_parameters_all.combine_first(other)

df_parameters_all.rename(columns={'coeff':'regressed'}, inplace=True)
df_parameters_all.drop(columns=['brix_lambda'], inplace=True)

df_parameters_all['SI'] = df_parameters_all['regressed'].combine_first(df_parameters_all['brix_SI'])  # prefer regressed
df_parameters_all['SI'].fillna(0,inplace=True)

In [ ]:
df_parameters_all.head()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,5)
mask = ((df_parameters_all['brix_prime'] != 0) |
        (df_parameters_all['regressed']).notnull())
        
df_parameters_plot = df_parameters_all.loc[mask]
df_parameters_plot.plot.bar(y=['brix_SI','regressed'], ax=ax);

# Simulate captive

In [ ]:
#subs = {value:key for key,value in eq.p.items()}
#X_qs_lambda_regress = lambdify(eq.X_qs_eq.rhs.subs(subs))
#Y_qs_lambda_regress = lambdify(eq.Y_qs_eq.rhs.subs(subs))
#N_qs_lambda_regress = lambdify(eq.N_qs_eq.rhs.subs(subs))

In [ ]:
df_all['X_qs_pred'] = run(function=simulation.X_qs_lambda, inputs=df_all, **df_parameters_all['SI'])
df_all['Y_qs_pred'] = run(function=simulation.Y_qs_lambda, inputs=df_all, **df_parameters_all['SI'])
df_all['N_qs_pred'] = run(function=simulation.N_qs_lambda, inputs=df_all, **df_parameters_all['SI'])


In [ ]:
df_ = runs.get_group(22774)

for key in ['X_qs','Y_qs','N_qs']:
    fig,ax=plt.subplots()
    df_.plot(x=key, y=f'{key}_pred',style='.', ax=ax, alpha=0.2)
    ax.grid(True)

track_plot(df=df_, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam);
df_.plot(y=['X_qs','X_qs_pred','thrust'])

fig,ax=plt.subplots()
df_.plot(x='t', y=['N_qs'], ax=ax)
df_.plot(x='t', y=['N_qs_pred'], alpha=0.5, ax=ax)
ax2 = ax.twinx()
df_.plot(x='t', y=['delta'], style='--', alpha=0.5, ax=ax2)
ax.grid(True)

ax.grid(True)

df_.plot(y=['u','v'])
df_.plot(y=['u1d','v1d'])

In [ ]:
mask = ((df_['N_qs'] > 6) & (df_['N_qs'] < 9))
df_.loc[mask].plot(x='t', y=['N_qs','N_qs_pred'], style='.')


fig,ax=plt.subplots()
df_.plot(x='t', y=['N_qs'], ax=ax)
df_.plot(x='t', y=['N_qs_pred'], style='.', alpha=0.5, ax=ax)
ax.set_xlim(0,3)

ax2 = ax.twinx()
df_.plot(x='t', y=['delta'], style='--', alpha=0.5, ax=ax2)
ax.legend(loc='lower right')
ax.grid(True)


fig,ax=plt.subplots()
df_.plot(x='t', y=['u','v','r'], ax=ax)
ax.set_xlim(0,1)

In [ ]:
X = diff_eq_N.calculate_features(data=df_)
y = diff_eq_N.calculate_label(y=df_['N_qs'])

N_forces = X*results_summary_N['coeff']
N_forces['t'] = df_['t']
#X_forces['id'] = df_all['id']
display(px.line(N_forces, x='t', y=N_forces.columns, width=1000, height=400))

In [ ]:
#df_.head()

In [ ]:
for key in ['X_qs','Y_qs','N_qs']:
    fig,ax=plt.subplots()
    
    for id,group in df_all.groupby(by='id'):
        group.plot(x=key, y=f'{key}_pred',style='.', ax=ax, label=id)
    
    ax.plot([df_all[key].min(), df_all[key].max()], [df_all[key].min(), df_all[key].max()], 'r-')
    
    #ax.get_legend().set_visible(False)

In [ ]:
df_captive = pd.DataFrame()

V_ = 2.0

betas = np.deg2rad(np.linspace(0,30,5))
deltas = np.deg2rad(np.linspace(-35,35,5))

Betas, Deltas = np.meshgrid(betas,deltas)

df_captive['beta'] = Betas.flatten()
df_captive['delta'] = Deltas.flatten()

df_captive['u'] = V_*np.cos(df_captive['beta'])
df_captive['v'] = -V_*np.sin(df_captive['beta'])
df_captive['thrust'] = 0
df_captive['r'] = 0


df_captive['X_qs_pred'] = run(function=simulation.X_qs_lambda, inputs=df_captive, **df_parameters_all['SI'])
df_captive['Y_qs_pred'] = run(function=simulation.Y_qs_lambda, inputs=df_captive, **df_parameters_all['SI'])
df_captive['N_qs_pred'] = run(function=simulation.N_qs_lambda, inputs=df_captive, **df_parameters_all['SI'])

keys = ['X_qs','Y_qs','N_qs']

x_keys = ['beta','delta']
for x_key in x_keys:
    
    fig,axes=plt.subplots(ncols=len(keys))
    fig.set_size_inches(15,4)
    
    others = list(x_keys)
    others.remove(x_key)
    
    for group_name, group in df_captive.groupby(others):
        
        if not isinstance(group_name,list):
            group_name = [group_name]
        label =''.join([f'{other}:{np.round(name,2)}, ' for other,name in zip(others,group_name)])
        
        for key,ax in zip(keys,axes):
            
            group.plot(x=x_key, y=f'{key}_pred',style='.-', ax=ax, label=label)
            #ax.get_legend().set_visible(False)
            ax.set_title(key)
            ax.grid(True)

# Simulation

In [ ]:
u1d,v1d,r1d = sp.symbols('u1d, v1d, r1d')
subs = [
    (u1d,u1d),
    (v1d,v1d),
    (r1d,r1d),

]
eq_X_ = X_eq.subs(subs)
eq_Y_ = Y_eq.subs(subs)
eq_N_ = N_eq.subs(subs)


A,b = sp.linear_eq_to_matrix([eq_X_,eq_Y_,eq_N_],[u1d,v1d,r1d])

acceleartion_eq = A.inv()*b
## Lambdify:
subs = {value:key for key,value in eq.p.items()}
subs[X_qs] = sp.symbols('X_qs')
subs[Y_qs] = sp.symbols('Y_qs')
subs[N_qs] = sp.symbols('N_qs')

acceleartion_lambda = lambdify(acceleartion_eq.subs(subs))

In [ ]:
A

In [ ]:
b

In [ ]:
from scipy.spatial.transform import Rotation as R

def step(t, states, parameters, ship_parameters, control):

    u,v,r,x0,y0,psi = states
    
    if u < 0:
        dstates = [
        0,
        0,
        0,
        0,
        0,
        0,
        ]    
        return dstates
    
    states_dict = {
        'u':u,
        'v':v,
        'r':r,
        'x0':x0,
        'y0':y0,
        'psi':psi,
        
        }
    
    inputs = dict(parameters)
    inputs.update(ship_parameters)
    inputs.update(states_dict)
    
    if isinstance(control, pd.DataFrame):
        index = np.argmin(np.array(np.abs(control.index - t)))
        control_ = dict(control.iloc[index])
    else:
        control_ = control
    inputs.update(control_)
    
    #inputs['U'] = np.sqrt(u**2 + v**2)  #Instantanious velocity
    
    inputs['X_qs'] = run(function=X_qs_lambda_regress, inputs=inputs)
    inputs['Y_qs'] = run(function=Y_qs_lambda_regress, inputs=inputs)
    inputs['N_qs'] = run(function=N_qs_lambda_regress, inputs=inputs)
    u1d,v1d,r1d = run(function=acceleartion_lambda, inputs=inputs)
    
    rotation = R.from_euler('z', psi, degrees=False)
    w = 0
    velocities = rotation.apply([u,v,w])
    x01d = velocities[0]
    y01d = velocities[1]
    psi1d = r    
    dstates = [
        u1d,
        v1d,
        r1d,
        x01d,
        y01d,
        psi1d,
    ]    
    return dstates

In [ ]:
def simulate(df_, parameters, ship_parameters):
    
    t = df_.index
    t_span = [t.min(),t.max()]
    t_eval = np.linspace(t.min(),t.max(),len(t))
    
    control = df_[['delta','thrust']]
    
    df_0 = df_.iloc[0:10].median(axis=0)
    y0 = {
        'u' : df_0['u'], 
        'v' : df_0['v'],
        'r' : df_0['r'],
        'x0' : df_0['x0'],
        'y0' : df_0['y0'],
        'psi' : df_0['psi']
        }
    
    solution = solve_ivp(fun=step, t_span=t_span, y0=list(y0.values()), t_eval=t_eval, 
                args=(parameters, ship_parameters, control))
    
    columns = list(y0.keys())
    df_result = pd.DataFrame(data=solution.y.T, columns=columns)
    df_result.index=t[0:len(df_result)]
    
    df_result['beta'] = -np.arctan2(df_result['v'],df_result['u'])
    
    return solution, df_result

In [ ]:
#id=22773
#id=22612
#id=22774
#id=22770

id = 22764 #	zigzag	10.0
#id = 22772 #	zigzag	20.0
#id = 22774 #	turning circle	NaN

parameters = df_parameters_all['SI'].copy()

df = runs.get_group(id).copy()
df.set_index('t', inplace=True)
simulation.simulate(df, parameters = parameters, 
                               ship_parameters=ship_parameters)
df_result = simulation.result

In [ ]:
fig,ax=plt.subplots()
track_plot(df=df, lpp=ship_parameters['L'], beam=ship_parameters['B'],ax=ax, label='model test')
track_plot(df=df_result, lpp=ship_parameters['L'], beam=ship_parameters['B'],ax=ax, label='simulation', color='green')
ax.legend()

for key in df_result:
    fig,ax = plt.subplots()
    df.plot(y=key, label='model test', ax=ax)
    df_result.plot(y=key, label='simulation', ax=ax)
    ax.set_ylabel(key)

In [ ]:
data = df_result
data['delta'] = df['delta']
data['thrust'] = df['thrust']

X = diff_eq_N.calculate_features(data=data)
N_forces = X*results_summary_N['coeff']
N_forces.index = df_result.index
#X_forces['id'] = df_all['id']
display(px.line(N_forces.loc[0:100], y=N_forces.columns, width=1000, height=400))